In [42]:
import csv
import os
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd

In [43]:
matrices_dir = "/Users/ndeveaux/Dropbox (Simons Foundation)/Drosophila_Inferelator_Project/network_inputs_and_outputs/outputs"

In [44]:
new_nets = ['2018-06-13_13-41-39',
'2018-06-15_05-25-53']

In [45]:
gtf = pd.read_csv('~/Dev/Drosophila_inferelator/Drosophila_melanogaster.BDGP6.91.genes.gtf', sep='\t', header = None)

In [46]:
gene_id_to_sym = {}
for (idx, row) in gtf.iterrows():
    st = row[8]
    splits = st.split(';')
    i = splits[0].split(" ")[1].strip('"')
    sym = splits[1].split(" ")[2].strip('"').rstrip(';').rstrip('"').replace(';', '')
    gene_id_to_sym[i] = sym

In [47]:
def get_symbol(fbgn_id):
    if fbgn_id in gene_id_to_sym.keys():
        return gene_id_to_sym[fbgn_id]
    return fbgn_id

In [48]:
def convert_net(net):
    to_gephi_net = net.copy()
    to_gephi_net['Target'] = [get_symbol(i) for i in net['target']]
    to_gephi_net['Source'] = [get_symbol(i) for i in net['regulator']]
    to_gephi_net = to_gephi_net.drop(['target', 'regulator'], axis=1)
    return to_gephi_net

In [49]:
old_nets = ['WtJ_and_WtF_KNN_Smoothed_data/2018-03-28_16-19-38', 'WtJ_and_WtF_KNN_Smoothed_data/2018-03-28_13-49-36', 'WtJ_and_WtF_KNN_Smoothed_data/2018-03-28_12-57-21']

In [50]:
net_dict = {}

In [51]:
for n in old_nets + new_nets:
    net = pd.read_csv(os.path.join(os.path.join(matrices_dir, n), 'network.tsv'), sep='\t')
    net_dict[n] = convert_net(net)

In [52]:
gephi_nets = ['cutoff_0.5_to_gephi_network.tsv', 'cutoff_0.7_to_gephi_network.tsv', 'cutoff_0.3_to_gephi_network.tsv', 'cutoff_0.9_to_gephi_network.tsv']

In [53]:
for g in gephi_nets:
    net = pd.read_csv(os.path.join(os.path.join(matrices_dir, 'dmel_expression_TF_filtered_experiment'), g), sep='\t')
    net_dict[g] = net

In [54]:
#add embryo
embryo_net_location = '/Users/ndeveaux/Dropbox (Simons Foundation)/Drosophila_Inferelator_Project/embryo_network/2018-05-04_14-31-31_no_smoothing/network.tsv'
net = pd.read_csv(embryo_net_location, sep = '\t')
net_dict['embryo'] = convert_net(net)

In [55]:
for n in net_dict:
    net = net_dict[n]
    max_beta = np.max(net['beta.non.zero'])
    cutoff = max_beta / 2.0
    filtered_net= net[net['beta.non.zero'] > cutoff ]
    ribos = [r for r in filtered_net['Target'] if r.startswith('Rp')]
    print('{} network:'.format(n))
    print('number of entries: {}'.format(filtered_net.shape[0]))
    print('number of ribosomal gene entries: {}'.format(len(ribos)))
    print('percentage of entries that are ribosomal: {0:.2f}%'.format(100*len(ribos) / filtered_net.shape[0]))
    print('50% beta cutoff: {}'.format(cutoff))
    print()
    

embryo network:
number of entries: 8242
number of ribosomal gene entries: 417
percentage of entries that are ribosomal: 5.06%
50% beta cutoff: 10.0

cutoff_0.3_to_gephi_network.tsv network:
number of entries: 642
number of ribosomal gene entries: 90
percentage of entries that are ribosomal: 14.02%
50% beta cutoff: 10.0

cutoff_0.5_to_gephi_network.tsv network:
number of entries: 522
number of ribosomal gene entries: 92
percentage of entries that are ribosomal: 17.62%
50% beta cutoff: 10.0

cutoff_0.9_to_gephi_network.tsv network:
number of entries: 348
number of ribosomal gene entries: 112
percentage of entries that are ribosomal: 32.18%
50% beta cutoff: 10.0

WtJ_and_WtF_KNN_Smoothed_data/2018-03-28_13-49-36 network:
number of entries: 5195
number of ribosomal gene entries: 139
percentage of entries that are ribosomal: 2.68%
50% beta cutoff: 30.0

2018-06-13_13-41-39 network:
number of entries: 6495
number of ribosomal gene entries: 454
percentage of entries that are ribosomal: 6.99%


In [56]:
# Save nets 
names = ['fly_factor_survey_TFs_to_gephi_network.tsv', 'Marbach_and_fly_factor_survey_TFs_to_gephi_network.tsv']
for idx, n in enumerate(new_nets):
    net_dict[n].to_csv(os.path.join(os.path.join(matrices_dir, n), names[idx]), sep='\t')
    

In [57]:
'Marbach_and_fly_factor_survey_to_gephi_network.tsv'

'Marbach_and_fly_factor_survey_to_gephi_network.tsv'

In [58]:
new_nets

['2018-06-13_13-41-39', '2018-06-15_05-25-53']